In [24]:
import os
import json

import openai
from dotenv import load_dotenv


load_dotenv("../.env")
openai.api_key = os.getenv("OPENAI_API_KEY")

In [33]:
# write function to get flights for a day, location and airline
functions =  [
  {
    "name": "book_best_flight",
    "description": "Will return confirmation for a flight booked given a departure city, arrival city, departure date, cabin class, and airline",
    "parameters": {
      "type": "object",
      "properties": {
        "departure_city": {
          "type": "string",
          "description": "The IATA code for the city of departure e.g. JFK, LHR, CDG"
        },
        "arrival_city": {
          "type": "string",
          "description": "The IATA code for the city of arrival e.g. JFK, LHR, CDG"
        },
        "departure_date": {
          "type": "string",
          "description": "The date of departure in the format YYYY-MM-DD"
        },
        "time_of_day": {
          "type": "string",
          "enum": ["morning", "afternoon", "evening"]
        },
        "cabin_class": {
          "type": "string",
          "enum": ["first", "business", "premium_economy", "economy"]
        },
        "airline": {
          "type": "string",
          "description": "The airline the user would like to fly in e.g. Delta, United, American Airlines"
        },
      },
      "required": ["departure_city", "arrival_city", "departure_date"]
    }
  }
]

messages = [
  {"role": "system", "content": "You're a friendly flight booking assistant. You will be given a user query. Do your best to parse the user's request and respond accordingly. If the user hasn't given you enough information to make a function call, ask them for clarification"}
]

In [34]:
def user_interaction(query: str): 
  messages.append({"role": "user", "content": query})
  response = openai.chat.completions.create(
    model="gpt-4-1106-preview",
    messages=messages,
    functions=functions,
    function_call="auto"
  )
  reason = response.choices[0].finish_reason
  if reason == "function_call":
    # handle function call
    return response.choices[0].message.function_call
  else:
    # return this output to user
    return response.choices[0].message.content

In [35]:
user_interaction("Hello, i'd like to book a flight from New York to Atlanta")

'Sure, I can help you with that. To proceed with the booking, I need some additional details:\n\n1. The date you would like to depart on.\n2. Your preferred time of day to fly (morning, afternoon, or evening).\n3. The class you would like to travel in (first class, business class, premium economy, or economy).\n4. If you have a specific flight number in mind, please provide that as well.\n\nCould you provide these details, please?'

In [36]:
call = user_interaction("I need a flight from london to paris")

In [25]:
args = json.loads(call.arguments)

{'departure_city': 'LHR',
 'arrival_city': 'CDG',
 'departure_date': '2023-04-27',
 'time_of_day': 'morning'}

In [37]:
call

"Sure, I'd be happy to help you book your flight from London to Paris. Could you please provide me with the following additional details?\n\n- Your preferred departure date\n- The time of day you’d like to depart (morning, afternoon, or evening)\n- Your cabin class preference (e.g., economy, business)\n- If you have a specific flight number in mind\n\nWith this information, I can proceed to book your flight."